# Create `openalex.works.work_author_affiliations_backfill`

Converts `openalex.authors.work_authorships_backfill_moderated` into a mapping table format.

In [ ]:
%run ../utils/variables

In [ ]:
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.work_author_affiliations_backfill') AS
WITH backfill_exploded AS (
    -- Explode authorships to get each author
    SELECT 
        paper_id AS work_id,
        author_sequence,
        authorship.raw_author_name,
        authorship.affiliations
    FROM openalex.authors.work_authorships_backfill_moderated
    LATERAL VIEW POSEXPLODE(authorships) t AS author_sequence, authorship
),
backfill_with_affiliations AS (
    -- Explode affiliations to get each affiliation per author
    SELECT 
        work_id,
        author_sequence,
        raw_author_name,
        affiliation.raw_affiliation_string
    FROM backfill_exploded
    LATERAL VIEW OUTER EXPLODE(affiliations) t AS affiliation
)
SELECT DISTINCT
    work_id,
    author_sequence,
    raw_author_name,
    raw_affiliation_string,
    current_timestamp() AS created_at
FROM backfill_with_affiliations;